In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from test_db_schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [4]:
from typing import Any


async def dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    match_by=User.name,
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                        "organization_countries": Organization.countries,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "organization_countries": ["USA", "Mexico", "Canada"],
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "organization_countries": ["USA", "Mexico"],
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "organization_countries": ["USA", "Canada"],
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [7]:
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(schema=TestSchema)
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

In [8]:
db[User].values()


[
    User({'name': 'John', 'age': 31, '_id': 'd97d9bce-618d-4f5e-bed2-8a1cfe5b8342'}),
    User({'name': 'Jane', 'age': 56, '_id': '1e43611d-4c87-457e-9720-922e1af9b870'}),
    User({'name': 'Jack', 'age': 66, '_id': '0b839301-17fe-4ac1-b672-9dc281dd3009'})
]

In [9]:
db[Search.term].values()

['test']

In [10]:
list(db[Search.result_count].items())

[('test', 3)]

In [11]:
list(db[Search.results].items())


[
    (
        ('test', 2),
        Project({'number': 2, 'name': 'fixing cars', 'start': datetime.date(2020, 1, 3), 'end': datetime.date(2020, 1, 6), 'status': 'started'})
    ),
    (
        ('test', 0),
        Project({'number': 0, 'name': 'baking cake', 'start': datetime.date(2020, 1, 1), 'end': datetime.date(2020, 1, 4), 'status': 'done'})
    ),
    (
        ('test', 1),
        Project({'number': 1, 'name': 'cleaning shoes', 'start': datetime.date(2020, 1, 2), 'end': datetime.date(2020, 1, 5), 'status': 'done'})
    )
]

In [12]:
list(db[Search.results].items())[0][1].start.year

2020

In [13]:
db[Search.results].df()

test_db_schema_Search_term,test_db_schema_Search_19c46efdd2_test_db_schema_Project_number,end,name,status,number,test_db_schema_Organization__id,start
str,i64,date,str,str,i64,str,date
"""test""",2,2020-01-06,"""fixing cars""","""started""",2,"""900ab5cb-4e21-4629-bc04-116c4d…",2020-01-03
"""test""",0,2020-01-04,"""baking cake""","""done""",0,"""b7288037-e0be-42e2-81a2-5df38b…",2020-01-01
"""test""",1,2020-01-05,"""cleaning shoes""","""done""",1,"""9e768c8f-f7c3-46cd-b6d7-f33b4e…",2020-01-02


In [14]:
db[Organization].df()

test_db_schema_Organization__id,city,address,name
str,str,str,str
"""b7288037-e0be-42e2-81a2-5df38b…","""Bakerville""","""Main Street 1""","""Bakery"""
"""900ab5cb-4e21-4629-bc04-116c4d…","""Shoetown""","""Main Street 2""","""Shoe Shop"""
"""9e768c8f-f7c3-46cd-b6d7-f33b4e…","""Cartown""","""Main Street 3""","""Car Shop"""


In [19]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'test_db_schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev133+gf6dad64.d20250116',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': 'df58919723980619ef601b878cdb788bdcd19e34',
        'module_path': WindowsPath('exp/test_db_schema.py'),
        'module_dirty': False,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'test_db_schema.Search': 1,
            'test_db_schema.Project': 3,
            'test_db_schema.Task': 6,
            'test_db_schema.Organization': 3,
            'test_db_schema.User': 3
        },
        'arrays': {'test_db_schema_Organization_countries': 7},
        'relations': {'test_db_schema.SearchResult': 3, 'test_db_schema.Membership': 7}
    },
    'backend': None
}

In [20]:
db[SearchResult._table[SearchResult.score.isin(slice(0.81, 0.9))]].df()

test_db_schema_SearchResult__id,score,test_db_schema_Search_term,test_db_schema_Project_number
str,f64,str,i64
"""6255212236""",0.9,"""test""",0


In [21]:
db[Organization.countries].df()

test_db_schema_Organization__id,test_db_schema_Organization_f68dd49e88__id,_value
str,i64,str
"""b7288037-e0be-42e2-81a2-5df38b…",0,"""USA"""
"""b7288037-e0be-42e2-81a2-5df38b…",1,"""Mexico"""
"""b7288037-e0be-42e2-81a2-5df38b…",2,"""Canada"""
"""9e768c8f-f7c3-46cd-b6d7-f33b4e…",0,"""USA"""
"""9e768c8f-f7c3-46cd-b6d7-f33b4e…",1,"""Canada"""
"""900ab5cb-4e21-4629-bc04-116c4d…",0,"""USA"""
"""900ab5cb-4e21-4629-bc04-116c4d…",1,"""Mexico"""


In [22]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         480950 function calls (446749 primitive calls) in 0.678 seconds

   Ordered by: cumulative time
   List reduced from 2272 to 158 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       56    0.000    0.000    0.225    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:838(load)
       52    0.000    0.000    0.218    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:697(_load_rec_from_item)
       52    0.001    0.000    0.216    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:572(_load_record)
   168/57    0.001    0.000    0.210    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:665(_load_records)
       19    0.000    0.000    0.210    0.011 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1450(__ior__)
       19    0.001    0.000    0.210    0.011 C:\Users\lworm\Repos\py-research\src\py_research\db\databas